In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15825, done.
remote: Counting objects: 100% (286/286), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 15825 (delta 151), reused 219 (delta 140), pack-reused 15539
Receiving objects: 100% (15825/15825), 14.71 MiB | 15.84 MiB/s, done.
Resolving deltas: 100% (10585/10585), done.


In [ ]:
%cd darknet

/content/darknet


In [ ]:
# libso avx ve opencv 1 yap
!make

In [ ]:
# cuda var mi yok mu icin:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
# bilgisayarimdan da /Users ... diyerek drive'imdan alabilirdim, ama bu da olur
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
# Yapacagimiz tum importlari burada tek bir hucrede tanimlamak daha mantikli, tekrar tekrar ugrasmamak icin :D
import glob
import numpy as np
import cv2
import darknet
import matplotlib.pyplot as plt

In [ ]:
network, class_names, colors = darknet.load_network("/mydrive/yolo/yolov4-obj.cfg","/mydrive/yolo/obj_copy.data","/mydrive/yolo/backup/yolov4-obj_last.weights")

In [ ]:
width = darknet.network_width(network)
height = darknet.network_height(network)

In [ ]:
print(colors)

{'dur': (231, 162, 125), 'durak': (150, 151, 220), 'parkyasak': (76, 84, 209), 'parkyeri': (185, 60, 113), 'sagadonulmez': (4, 231, 115), 'soladonulmez': (122, 120, 36), 'ilerisag': (245, 27, 112), 'ilerisol': (114, 99, 82), 'sagmecburi': (106, 96, 192), 'solmecburi': (35, 150, 189), 'girisyok': (46, 1, 161), 'hizsiniri20': (43, 229, 219), 'hizsiniri30': (112, 165, 229), 'kirmiziisik': (197, 41, 49), 'sariisik': (116, 5, 84), 'yesilisik': (145, 212, 255)}


In [ ]:
# yardimci fonksiyonlardan aldim, foto gorsellestirmek istersek kullaniriz
def imShow(path):
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# image path'imizdeki gorselimizi darknet'e uygun bir image haline getirme
def opencv_demo(image_path):
  image = cv2.imread(image_path) # Görüntüyü OpenCV ile okuyoruz
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # opencv bgr formatinda aliyor goruntuyu, biz bunu rgb formatina ceviriyoruz, bunu yapmadan once dogruluk 70'lerdeydi
  resized_image = cv2.resize(image_rgb, (width, height),interpolation=cv2.INTER_LINEAR) # Görüntüyü Darknet için belirtilen boyutlara yeniden boyutlandırıyoruz
  darknet_image = darknet.make_image(width, height, 3) # Darknet için uygun bir görüntü formatına dönüştüruyoruz
  darknet.copy_image_from_bytes(darknet_image, resized_image.tobytes())
  return darknet_image

In [ ]:
def main(klasor_yolu, class_name):
  jpg_dosyalari = glob.glob(f'{klasor_yolu}/*.jpg') # Klasör içindeki tüm JPG dosyalarını buluyoruz
  a,b,d = 0,0,0
  for jpg_path in jpg_dosyalari: # JPG dosyalarına döngüyle erişiyoruz
      # print(jpg_path)
    darknet_image = opencv_demo(jpg_path)
    detection = darknet.detect_image(network, class_names, darknet_image, thresh=0.65, hier_thresh=0.5, nms=0.45)
    # print(detection)
    c = 0
    for i in detection:
      c += 1
    if c == 1:
      for isim, dogruluk,(x, y, w, h) in detection:
        if isim == class_name:
          a += 1
          # print(f'{isim} {dogruluk}')
        elif isim != class_name:
          b += 1
          # print(f'{isim} {dogruluk}')
    else:
      d += 1

  print(f'-----{class_name}-----\nDogru Tahmin: {a}, Yanlis Tahmin: {b}\nBirden fazla class tahmini: {d}\n')

In [ ]:
main('/mydrive/yolo/sagadonulmez','sagadonulmez') # toplam 102 tane foto
main('/mydrive/yolo/soladonulmez','soladonulmez') # toplam 245 tane foto
main('/mydrive/yolo/sagmecburi','sagmecburi') # toplam 49 tane foto
main('/mydrive/yolo/solmecburi','solmecburi') # toplam 34 tane foto
main('/mydrive/yolo/ilerisag','ilerisag') # toplam 121 tane foto
main('/mydrive/yolo/ilerisol','ilerisol') # toplam 99 tane foto  # hepsi toplam 650

-----sagadonulmez-----
Dogru Tahmin: 61, Yanlis Tahmin: 18
Birden fazla class tahmini: 23

-----soladonulmez-----
Dogru Tahmin: 210, Yanlis Tahmin: 19
Birden fazla class tahmini: 16

-----sagmecburi-----
Dogru Tahmin: 47, Yanlis Tahmin: 0
Birden fazla class tahmini: 2

-----solmecburi-----
Dogru Tahmin: 30, Yanlis Tahmin: 2
Birden fazla class tahmini: 2

-----ilerisag-----
Dogru Tahmin: 80, Yanlis Tahmin: 12
Birden fazla class tahmini: 29

-----ilerisol-----
Dogru Tahmin: 65, Yanlis Tahmin: 12
Birden fazla class tahmini: 22



In [ ]:
# emre abi ile yazdik :D
def main(general_path="/mydrive/yolo/"): # main fonksiyonu main() ya da main(path) seklinde kullanilir
    aranan_tabelalar = ["sagadonulmez",
                        "soladonulmez",
                        "sagmecburi",
                        "solmecburi",
                        "ilerisag",
                        "ilerisol"
                        ]
    array = np.zeros((7, 6))
    for test_edilen_tabela in aranan_tabelalar:
        jpg_dosyalari = glob.glob(general_path + test_edilen_tabela + "/*.jpg")
        for jpg_path in jpg_dosyalari:
            darknet_image = opencv_demo(jpg_path)
            detection = darknet.detect_image(network, class_names, darknet_image, thresh=0.65, hier_thresh=0.5, nms=0.45)
            counter = 0
            for isim, _, _ in detection:
                if isim in aranan_tabelalar:
                    counter += 1
                    if isim == test_edilen_tabela:
                        array[aranan_tabelalar.index(test_edilen_tabela), aranan_tabelalar.index(test_edilen_tabela)] += 1
                    else:
                        array[aranan_tabelalar.index(isim), aranan_tabelalar.index(test_edilen_tabela), ] += 1
                else:
                    pass
            if counter == 0:
              array[6,aranan_tabelalar.index(test_edilen_tabela)] += 1
    return array

In [ ]:
array = main()
print(array) # en alt satir recall oluyor, tespit edilememe sayilari

[[ 53.  10.   1.   1.   0.   0.]
 [ 15. 210.   0.   0.   0.   0.]
 [  0.   0.  46.   1.   1.   0.]
 [  0.   0.   0.  31.   0.   0.]
 [  0.   0.   0.   0.  82.   9.]
 [  0.   0.   0.   1.  15.  67.]
 [ 36.  25.   3.   1.  23.  23.]]


In [ ]:
a = np.sum(array.reshape(-1))
print(a) # normalde 650 foto oldugu icin 650 donmeliydi ama 4 tane fazlalik goz cikarmaz :D

654.0


In [ ]:
np.save('confusion_matrix.npy', array) # zaten save edip indiridim ben bilgisayara, array'in tasinimi kolay olsun diye .npy seklinde tutuyoruz sonrasinda np.load('dosya_yolu') seklinde array degiskenine atayip kullanabiliyoruz

In [32]:
cap = cv2.VideoCapture('/mydrive/yolo/bariyersiz_parkur2.mp4')
from google.colab.patches import cv2_imshow
while True:
    ret, frame = cap.read()  # Bir kareyi yakala
    if not ret:
        break  # Videonun sonuna geldiyseniz döngüyü kır

    # İşleme adımlarını burada yapabilirsiniz

    # cv2_imshow(frame)  # İşlenmiş kareyi göster

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # 'q' tuşuna basarak döngüyü sonlandır
